In [1]:
import numpy as np
import pandas as pd
import scipy.io
import h5py

In [2]:
# downloaded from https://data.mendeley.com/datasets/p7ft2bvphx/1
v1 = scipy.io.loadmat('data/empirical/SPKcounts_V20160929cue_MW_250X250ms.mat')
v2 = scipy.io.loadmat('data/empirical/SPKcounts_V20160930cue_MW_250X250ms.mat')
v3 = scipy.io.loadmat('data/empirical/SPKcounts_V20161005cue_MW_250X250ms.mat')
v4 = scipy.io.loadmat('data/empirical/SPKcounts_V20161017cue_MW_250X250ms.mat')
w1 = scipy.io.loadmat('data/empirical/SPKcounts_W20160112cue_MW_250X250ms.mat')
w2 = scipy.io.loadmat('data/empirical/SPKcounts_W20160113cue_MW_250X250ms.mat')
w3 = scipy.io.loadmat('data/empirical/SPKcounts_W20160121cue_MW_250X250ms.mat')
w4 = scipy.io.loadmat('data/empirical/SPKcounts_W20160122cue_MW_250X250ms.mat')
vs = [v1, v2, v3, v3]
ws = [w1, w2, w3, w4]

In [3]:
# columns = ['monkey', 'session', 'block', 'trial', 'va', 'vb', 'vl', 'vr', 'w', 'al', 'ar', 'dw', 'dvc', 'dvs', 'dva'] 
# columns = ['monkey', 'session', 'block', 'trial'] 
labels = [
    "DeltaRel",
    "omega",
    "omega_BlockMean",
    "DeltaOmega",
    "pLeft",
    "pChosen",
    "V_Stim_C",
    "V_Act_C",
    "UpdatedV_Stim_C",
    "UpdatedV_Act_C",
    "RPE_Stim",
    "RPE_Action",
    "RPE_combined",
    "V_Stim_U",
    "V_Act_U",
    "V_Stim1",
    "V_Stim2",
    "V_Act1",
    "V_Act2",
    "choseBetterStim",
    "choseBetterAct",
    "V_StimR",
    "V_StimL",
    "V_Act1_End",
    "V_Act2_End",
    "V_StimR_End",
    "V_StimL_End",
    "V_StimPref",
    "V_StimUnpref",
    "DV_cho",
    "DVcho_Stim",
    "DVcho_Action",
    "DV_left",
    "DV_right",
    "EffOmega",
    "beta_Stim_t",
    "beta_Action_t",
    "prevChoseBetterStim",
    "prevChoseBetterAct",
    "DeltaV_Stim",
    "DeltaV_Act",
    "DeltaV_Stim_End",
    "DeltaV_Act_End"]

targets = ['V_StimL', 'V_StimR', 'V_Act1', 'V_Act2', 'DeltaV_Stim', 'DeltaV_Act', 'DV_left', 'DV_right', 'omega',
           'choseBetterStim', 'choseBetterAct', ]
target_idxs = [i for i, item in enumerate(labels) if item in targets]

for monkey in ['V', 'W']:
    for session in range(4):
        print(f"monkey {monkey} session {session}")
        sess = 1+session if monkey=='V' else 1+session+4
        files = vs if monkey=='V' else ws
        trial_data = files[session]['Y']
        bid = trial_data[:,7]
        tid = trial_data[:,5]
        letters = trial_data[:,0]
        locations = trial_data[:,1]
        rewards = trial_data[:,2]
        rl_vars = scipy.io.loadmat(f'data/matlab/all_vars_sess{sess}.mat')['AllVars'][0][0]
        for t in range(min([len(tid), 1920])):
            block = bid[t]
            trial = tid[t]
            letter = 1 if letters[t]==1 else -1
            location = 1 if locations[t]==1 else -1
            reward = 1 if rewards[t]==1 else -1
            # accuracy = 1 if ((block<=12 & rl_vars[19][t]==1) or (block>12 & rl_vars[20][t]==1)) else 0  # 19 stim, 20 act
            df_list = [monkey, session, block, trial, letter, location, reward]
            label_list = ['monkey', 'session', 'block', 'trial', 'chosen letter', 'chosen location', 'reward']
            for i in target_idxs:
                values = rl_vars[i].flatten()
                label = labels[i]
                if t<len(values):
                    value = values[t]
                    df_list.append(value)
                    label_list.append(label)
            # file = f"data/emp_spikes/monkey{monkey}_session{session}_block{block}_trial{trial}_values.pkl"
            file = f"data/rl/monkey{monkey}_session{session}_block{block}_trial{trial}_values.pkl"
            df = pd.DataFrame([df_list], columns=label_list)
            df.to_pickle(file)

monkey V session 0
monkey V session 1
monkey V session 2
monkey V session 3
monkey W session 0
monkey W session 1
monkey W session 2
monkey W session 3


In [4]:
# Map labels to target columns
labels = [
    "DeltaRel", "omega", "omega_BlockMean", "DeltaOmega", "pLeft", "pChosen",
    "V_Stim_C", "V_Act_C", "UpdatedV_Stim_C", "UpdatedV_Act_C",
    "RPE_Stim", "RPE_Action", "RPE_combined", "V_Stim_U", "V_Act_U",
    "V_Stim1", "V_Stim2", "V_Act1", "V_Act2", "choseBetterStim",
    "choseBetterAct", "V_StimR", "V_StimL", "V_Act1_End", "V_Act2_End",
    "V_StimR_End", "V_StimL_End", "V_StimPref", "V_StimUnpref", "DV_cho",
    "DVcho_Stim", "DVcho_Action", "DV_left", "DV_right", "EffOmega",
    "beta_Stim_t", "beta_Action_t", "prevChoseBetterStim", "prevChoseBetterAct",
    "DeltaV_Stim", "DeltaV_Act", "DeltaV_Stim_End", "DeltaV_Act_End"
]

targets = ['V_StimL', 'V_StimR', 'V_Act1', 'V_Act2', 'DeltaV_Stim', 'DeltaV_Act',
           'DV_left', 'DV_right', 'omega', 'choseBetterStim', 'choseBetterAct']

target_idxs = [i for i, item in enumerate(labels) if item in targets]

for monkey in ['V', 'W']:
    for session in range(4):
        print(f"monkey {monkey} session {session}")
        sess = 1+session if monkey=='V' else 1+session+4
        files = vs if monkey=='V' else ws
        trial_data = files[session]['Y']
        bid = trial_data[:,7]
        tid = trial_data[:,5]
        letters = trial_data[:,0]
        locations = trial_data[:,1]
        rewards = trial_data[:,2]
        rl_vars = scipy.io.loadmat(f'data/matlab/all_vars_sess{sess}.mat')['AllVars'][0][0]

        # Group trials by block
        blocks_dict = {}
        for t in range(min(len(tid), 1920)):
            block = bid[t]
            trial = tid[t]
            if block not in blocks_dict:
                blocks_dict[block] = {}
            df_list = [monkey, session, block, trial]
            label_list = ['monkey', 'session', 'block', 'trial']

            # chosen letter/location/reward
            letter = 1 if letters[t]==1 else -1
            location = 1 if locations[t]==1 else -1
            reward = 1 if rewards[t]==1 else -1
            df_list.extend([letter, location, reward])
            label_list.extend(['chosen letter', 'chosen location', 'reward'])

            # target RL variables
            for i in target_idxs:
                values = rl_vars[i].flatten()
                label = labels[i]
                value = values[t] if t < len(values) else np.nan
                df_list.append(value)
                label_list.append(label)

            # Store as DataFrame per trial
            blocks_dict[block][trial] = pd.DataFrame([df_list], columns=label_list)

        # Save entire block to one file
        for block, trial_dfs in blocks_dict.items():
            # Concatenate all trials in this block
            df_block = pd.concat([trial_dfs[trial] for trial in sorted(trial_dfs.keys())], ignore_index=True)
            file_block = f"data/rl/monkey{monkey}_session{session}_block{block}_values.pkl"
            df_block.to_pickle(file_block)
            print(f"  Saved block {block} with {len(df_block)} trials")

monkey V session 0
  Saved block 19 with 80 trials
  Saved block 23 with 80 trials
  Saved block 24 with 80 trials
  Saved block 14 with 80 trials
  Saved block 2 with 80 trials
  Saved block 18 with 80 trials
  Saved block 12 with 80 trials
  Saved block 9 with 80 trials
  Saved block 17 with 80 trials
  Saved block 10 with 80 trials
  Saved block 3 with 80 trials
  Saved block 1 with 80 trials
  Saved block 6 with 80 trials
  Saved block 16 with 80 trials
  Saved block 11 with 80 trials
  Saved block 13 with 80 trials
  Saved block 22 with 80 trials
  Saved block 7 with 80 trials
  Saved block 20 with 80 trials
  Saved block 15 with 80 trials
  Saved block 8 with 80 trials
  Saved block 4 with 80 trials
  Saved block 21 with 80 trials
  Saved block 5 with 80 trials
monkey V session 1
  Saved block 17 with 80 trials
  Saved block 3 with 80 trials
  Saved block 15 with 80 trials
  Saved block 21 with 80 trials
  Saved block 16 with 80 trials
  Saved block 20 with 80 trials
  Saved bloc